In [1]:
!pip install nltk
!pip install spacy
!pip  install pyLDAvis
!pip install tqdm
!python3 -m spacy download en

     |████████████████████████████████| 1.6MB 2.4MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=104ec9f85d1a262c8aee586e24b61b076b0431b5feaf6d6158d5c636f4929c2e
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
import tqdm
import nltk; nltk.download('stopwords')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
from collections import Counter

import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [3]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:
# Convert to list
data = df.content.values.tolist()


#YOURCODE

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [5]:
#tokenize

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [6]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [9]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 5), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [10]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bricklin', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('could', 1),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('where', 1),
  ('wonder', 1),
  ('year', 1)]]

In [11]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip mallet-2.0.8.zip
mallet_path = '/content/mallet-2.0.8/bin/mallet'

--2020-10-20 19:46:09--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  18.4MB/s    in 0.8s    

2020-10-20 19:46:10 (18.4 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Archive:  mallet-2.0.8.zip
   creating: mallet-2.0.8/
   creating: mallet-2.0.8/bin/
  inflating: mallet-2.0.8/bin/classifier2info  
  inflating: mallet-2.0.8/bin/csv2classify  
  inflating: mallet-2.0.8/bin/csv2vectors  
  inflating: mallet-2.0.8/bin/mallet  
  inflating: mallet-2.0.8/bin/mallet.bat  
  inflating: mallet-2.0.8/bin/mallethon  
  inflating: mallet-2.0.8/bin/prepend-license.sh  
  inflating: mallet-2.0.8/bin/svmlight2vectors  
  inflating: mallet-2.0.8/bin/text2classi

# Найдем оптимальное количество топиков

In [12]:
def find_best_num(dictionary, corpus, texts, limit, start, step):
    nums = []
    coherence = []
    for num_topics in range(start, limit, step):
      nums.append(num_topics)
      model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
      coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
      coherence.append(coherencemodel.get_coherence())
    return nums, coherence

In [13]:
topic_num, coherence_values = find_best_num(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=5, limit=31, step=6)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


# Обучим модель 

In [14]:
def optimal_topic(topic_num, coherence_values):
  best_num = {}
  for i in range(len(topic_num)):
    best_num[topic_num[i]] = coherence_values[i]
  return Counter(best_num).most_common(1)[0][0], Counter(best_num).most_common(1)[0][1]

In [15]:
optimal_topic_num, best_coh = optimal_topic(topic_num, coherence_values)
print(optimal_topic_num, best_coh)

29 0.5457769411310336


In [16]:
model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=optimal_topic_num, id2word=id2word)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:252: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [17]:
topics_dict = {} # словарь топиков, в значении - словарь со словом и его весом
for topic in model.show_topics(formatted=False, num_topics=optimal_topic_num):
  topic_keywords = {}
  for key_word in topic[1]:
    topic_keywords[key_word[0]] = key_word[1]
  topics_dict[topic[0]] = topic_keywords

In [18]:
def get_main_topic(text, topics_dict):
  score = {}
  for i in range(optimal_topic_num):
    score[i] = 0
  for topic in topics_dict.keys():
    topic_words = list(topics_dict[topic].keys())
    for word in text:
      if word in topic_words:
        score[topic] += topics_dict[topic][word]
  return Counter(score).most_common(1)[0][0]

In [19]:
main_topics = {}
for i, text in enumerate(data_lemmatized):
  main_topics[i] = get_main_topic(text, topics_dict)

In [20]:
df = pd.DataFrame()
topic_words_to_df = []
for i in list(main_topics.values()):
  topic_words_to_df.append(list(dict(model.show_topic(i)).keys()))
df['text'] = data_lemmatized
df['main_topic'] = list(main_topics.values())
df['topic_word'] = topic_words_to_df

In [21]:
df

,text,main_topic,topic_word
0,"[where, thing, car, nntp_poste, host, park, li...",6,"[write, line, article, host, organization, nnt..."
1,"[poll, final, call, summary, final, call, cloc...",6,"[write, line, article, host, organization, nnt..."
2,"[engineering, computer, network, distribution_...",6,"[write, line, article, host, organization, nnt..."
3,"[division, line, host, write, write, article, ...",6,"[write, line, article, host, organization, nnt..."
4,"[question, distribution, article, write, clear...",6,"[write, line, article, host, organization, nnt..."
...,...,...,...
11309,"[scan, city, reply, line, consultation, cheap,...",6,"[write, line, article, host, organization, nnt..."
11310,"[screen, medford, old, problem, screen, blank,...",4,"[problem, card, line, driver, window, work, mo..."
11311,"[este, mount, mail, group, line, instal, try, ...",6,"[write, line, article, host, organization, nnt..."
11312,"[line, article, write, boy, embarasse, trivial...",6,"[write, line, article, host, organization, nnt..."


# Tf-IDF

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()

In [24]:
grouped = df.groupby(['main_topic'])['text']
all_grouped = []
for topic_id in set(df['main_topic']):
  for sent in grouped.get_group(topic_id).to_list():
    work_str = ''
    for word in sent:
      one_text_list = []
      work_str += word + ' '
      one_text_list.append(work_str)
    all_grouped.append(one_text_list)

In [35]:
def get_top_words(all_grouped):
  topic_key_words = []
  for group in all_grouped:
    tfIdfVectorizer=TfidfVectorizer(use_idf=True)
    tfIdf = tfIdfVectorizer.fit_transform(group)
    data = pd.DataFrame(tfIdf[0].T.todense(), index=tfIdfVectorizer.get_feature_names(), columns=["TF-IDF"])
    data = data.sort_values('TF-IDF', ascending=False)
    topic_key_words.append(list(data.index[:5]))
  return topic_key_words

In [36]:
df['keywords'] = get_top_words(all_grouped)

In [38]:
df.to_csv('output.csv')

In [37]:
df

,text,main_topic,topic_word,keywords
0,"[where, thing, car, nntp_poste, host, park, li...",6,"[write, line, article, host, organization, nnt...","[access, boot, list, control, terminal]"
1,"[poll, final, call, summary, final, call, cloc...",6,"[write, line, article, host, organization, nnt...","[file, print, name, printer, think]"
2,"[engineering, computer, network, distribution_...",6,"[write, line, article, host, organization, nnt...","[ice, afew, come, fish, know]"
3,"[division, line, host, write, write, article, ...",6,"[write, line, article, host, organization, nnt...","[platform, file, use, would, control]"
4,"[question, distribution, article, write, clear...",6,"[write, line, article, host, organization, nnt...","[byte, file, select, display, report]"
...,...,...,...,...
11309,"[scan, city, reply, line, consultation, cheap,...",6,"[write, line, article, host, organization, nnt...","[ms_myer, think, option, go, work]"
11310,"[screen, medford, old, problem, screen, blank,...",4,"[problem, card, line, driver, window, work, mo...","[claim, job, include, cult, system]"
11311,"[este, mount, mail, group, line, instal, try, ...",6,"[write, line, article, host, organization, nnt...","[say, know, think, go, president]"
11312,"[line, article, write, boy, embarasse, trivial...",6,"[write, line, article, host, organization, nnt...","[go, say, know, president, make]"


# Coherence score
Topic coherence оценивает семантическое сходство между самыми важными слоавми внутри темы.
Coherence score вычисляется с помощью двух метрик: внутренняя UMass и внешняя UCI, обе основаны на одной и той же идее, рассчитываются с помощью логарифмического распределения. Для подсчета внутренней метрики необходимо иметь в качестве данных последовательность (упорядоченную), так как эта метрика оценивает связь слова с соседними спереди и сзади. Внешняя метрика работает по другому. Считая ее мы сравниваем каждое слово с каждым.

http://qpleple.com/topic-coherence-to-evaluate-topic-models/
https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0#:~:text=But%20before%20that%E2%80%A6-,What%20is%20topic%20coherence%3F,are%20artifacts%20of%20statistical%20inference.
